# ME555 – Applications in Data and Materials Science
## Module - Homework

- Name: 
- NetID:

__Note__: After you filled out this notebook, go to `File` > `Save and Export Notebook As ...` > `PDF`  to save your results as PDF. Also save the notebook itself with `File` > `Download`. Rename both files to `Name_Firstname.pdf` and `Name_Firstname.ipynb` and send them per email to: [hagen.eckert@duke.edu](mailto:hagen.eckert@duke.edu).

In [ ]:
# Initialize imports and helper functions
from pathlib import Path
import json
from json import JSONDecodeError
from urllib.request import urlopen
from matplotlib import pyplot as plt

SERVER="https://aflow.org"
API="/API/aflux/?"

# Download a AFLUX response and return it as list of dictionaries
def aflux_request(matchbook, paging=1, no_directives=False):
    request_url = SERVER + API + matchbook
    if not no_directives:
        request_url += f",$paging({paging}),format(json)"
    server_response = urlopen(request_url)
    response_content  = server_response.read().decode("utf-8")
    # Basic error handling
    if server_response.getcode() == 200:
        try:
            return json.loads(response_content)
        except JSONDecodeError:
            pass
    print("AFLUX request failed!")
    print(f"  URL: {request_url}")
    print(f"  Response: {response_content}")
    return []

# Get CONTCAR file
def aflux_get_contcar(entry):
    request_url = "http://" + entry['aurl'].replace(':', '/') + '/' + 'CONTCAR.relax'
    server_response = urlopen(request_url)
    response_content  = server_response.read().decode("utf-8")
    if server_response.getcode() == 200:
        # Fix POSTCAR if in VASP4 format
        poscar_lines = response_content.split('\n')
        # Add species names if missing
        if poscar_lines[5].strip()[0].isnumeric:
            poscar_lines.insert(5, " ".join(entry['species']))
        poscar = '\n'.join(poscar_lines)
        return poscar
    print("AFLUX request failed!")
    print(f"  URL: {request_url}")
    print(f"  Response: {response_content}")
    return None

# Print the build in help of AFLUX
def aflux_help(keyword=None):
    if keyword is None:
        # General help (https://aflow.org/API/aflux/?)
        help_data = aflux_request("", no_directives=True)
        print("\n".join(help_data))
    else:
        # Help regarding a specific keyword (https://aflow.org/API/aflux/?help(keyword))
        help_data = aflux_request(f"help({keyword})")
        for key, entry in help_data.items():
            print(key)
            print(f"  description: {entry['description']}")
            print(f"  units: {entry['units']}")
            print(f"  status: {entry['status']}")
            comment = "\n    ".join(entry["__comment__"]).strip()
            if comment:
                print(f"  comment:\n    {comment}")

# Preload ML prediction results dictonary auid:ml_ael_bulk_modulus_vrh
prediction_data = json.loads(Path('prediction.json').read_text())

#### 1. Use AFLUX to retrieve all binary (two species) materials with a transition metal and a chalcogen.
   
a) How many of these compounds are metallic (`Egap_type = metal`)?   

b) Determine the bulk moduli (`ael_bulk_modulus_vrh`) of all transition metal chalcogenides with a band gap between 1 eV and 3 eV. For the five compounds with the highest bulk moduli, list the chemical formulas, space groups, bulk moduli, and band gaps.

c) Sort the transition metal chalcogenides by their magnetic moment (`spin_atom`) and determine the five compounds with the largest moments. List their chemical formulas, space groups, and magnetic moments.

d) Narrow the choice of transition metals down to `Fe`, `Ni`, and `Co`. Determine the five compounds with the largest magnetic moments and list their chemical formulas, space groups, and magnetic moments.

#### 2. AFLOW ML 
a) Determine the five highest/lowest bulk moduli (`ael_bulk_modulus_vrh`) of all compounds that contain Be. Already include the band gap (`Egap`) in your request.

b) Compare the bulk moduli and band gaps (`Egap`) of the best and worst compounds with results obtained from the PLMF machine learning model. To do this get the CONTCAR form AFLUX (`s`) and submit it to the [AFLOW ML tool](https://aflow.org/aflow-ml/).

- best system: 
  - Egap: 
  - ML Egap:
  - ael_bulk_modulus_vrh:
  - ML ael_bulk_modulus_vrh:
- worst system:
  - Egap: 
  - ML Egap:
  - ael_bulk_modulus_vrh:
  - ML ael_bulk_modulus_vrh:

#### 3. Use AFLUX to determine the total number of entries available for the Mo-Ti binary alloy system.

a) Find the formation enthalpies (`enthalpy_formation_atom`) for all entries with the 50:50 composition (`stoichiometry(‘0.5,0.5’)`) and use the bulk moduli results of the PLMF machine learning model stored in `prediction_data` to plot their distribution as a histogram.

b) Make a plot of the formation enthalpies versus the bulk moduli for the 50:50 composition. What is the relationship (if any) between the two quantities?